# GAN

## <코드1> 라이브러리 및 데이터 불러오기

### Ref : https://dreamgonfly.github.io/blog/gan-explained/#gan-직접-만들어보기

In [32]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

In [3]:
#데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
  transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로바꾼다.
  transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

#MNIST 데이터셋을 불러온다. 지정한 폴더에 없을 경우 자동으로 다운로드한다.
mnist =datasets.MNIST(root='dataset', download=True, transform=transform)

#데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
dataloader =DataLoader(mnist, batch_size=60, shuffle=True)


GAN의 2가지 요소인 생성자와 구분자 중 생성자(Generator)를 먼저 만들어보자. 생성자는 랜덤 벡터 ‘z’를 입력으로 받아 가짜 이미지를 출력하는 함수다. 여기서 ‘z’는 단순하게 균등 분포(Uniform Distribution)나 정규 분포(Normal Distribution)에서 무작위로 추출된 값이다. 생성자는 이렇게 단순한 분포를 사람 얼굴 이미지와 같은 복잡한 분포로 매핑(Mapping)하는 함수라고 볼 수 있다. 생성자 모델에 충분한 수의 매개 변수가 있다면 어떤 복잡한 분포도 근사할 수 있다는 것이 알려져 있다.

![](https://lens.google.com/search?ep=gsbubb&hl=ko&re=df&p=Acn1BYcI4aRioulCGXQkQr1HS88i8-_c-JCYFrykNJ28VitEqACmwjlIYAaCISug28cP6PX-krfDzfkOWIhRDb9otivAu5iNMblCogly19kMuGXBb78-PIzMlkYjPmsyV_kWc8aQGBYCunsCYHRSeS0FkD_cEDhkK8k2m7AF6IdWSl7ViTTO38iu7CNOWp2Zk-ajtpWF4FYSPciNFCPOCwQp#lns=W251bGwsbnVsbCxudWxsLG51bGwsbnVsbCxudWxsLG51bGwsIkVrY0tKREV3TXprd01HVXhMVEkxWkRjdE5EUXpNaTA1TXpkakxUY3hPREV6WmpsaE1HTmxNUklmU1hsdE1FRkVSMnhRUkhkUmQwMURZVjlYYUdaVlQxVjFTMmQxWld0U1p3PT0iXQ==)

‘z’ 벡터가 존재하는 공간을 잠재 공간(Latent Space)이라고도 부른다. 여기서는 잠재 공간의 크기를 임의로 100차원으로 뒀다. 잠재 공간의 크기에는 제한이 없으나 나타내려고 하는 대상의 정보를 충분히 담을 수 있을 만큼은 커야 한다. GAN은 우리가 이해할 수는 없는 방식이지만 ‘z’ 벡터의 값을 이미지의 속성에 매핑시키기 때문이다. 뒤에 살펴볼 GAN의 파생 모델에서 잠재 공간의 의미를 더욱 자세히 이해할 수 있을 것이다.

생성자에 충분한 수의 매개 변수를 확보하기 위해, 이 구현에서는 4개의 선형 레이어(Linear Layer, Fully Connected Layer, Linear Transformation)를 쌓아서 생성자를 만들었다. 선형 레이어는 속해있는 모든 뉴런이 이전 레이어의 모든 뉴런과 연결되는 가장 단순한 구조의 레이어다. 이 모델에서는 100차원의 랜덤 벡터를 받아 이를 256개의 뉴런을 가진 레이어로 보내고, 다시 레이어의 크기를 512, 1024로 점점 증가시켰다. 마지막에는 출력을 MNIST 이미지의 크기로 맞추기 위해 레이어 크기를 28x28로 줄였다.

각 레이어마다 활성 함수로는 LeakyReLU를 이용했다. LeakyReLU는 각 뉴런의 출력값이 0보다 높으면 그대로 놔두고, 0보다 낮으면 정해진 작은 숫자를 곱하는 간단한 함수다. 여기서는 0.2를 곱했다. 이밖에도 활성 함수로는 ReLU, Elu, Tanh, Sigmoid 등이 자주 쓰인다. 생성자의 마지막 레이어에서는 출력값을 픽셀값의 범위인 -1과 1 사이로 만들어주기 위해 Tanh를 사용했다.

이렇게 여러 개의 레이어와 활성 함수를 쌓은 덕분에 MNIST의 데이터 분포를 근사할 수 있는 충분한 표현력(Representation Power)을 얻을 수 있었다. MNIST는 비교적 간단한 문제에 속하므로 더욱 복잡한 문제를 풀기 위해서는 더 깊은 레이어 구조와 더 많은 양의 매개 변수가 필요할 것이다.

## # <코드2> GAN의 생성자(Generator)


In [4]:
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.
class Generator(nn.Module):
    # 네트워크 구조
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=100, out_features=256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=256, out_features=512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=512, out_features=1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(in_features=1024, out_features=28 * 28),
            nn.Tanh()
        )
    # (batch_size x 100) 크기의 랜덤 벡터를 받아 이미지를 (batch_size x 1 x 28 x 28)크기로 출력한다.
    def forward(self, inputs):
        return self.main(inputs).view(-1, 1, 28, 28)

구분자는 이미지를 입력으로 받고 그 이미지가 진짜일 확률을 0과 1 사이의 숫자 하나로 출력하는 함수다. 구분자의 구현은 생성자와 마찬가지로 4개의 선형 레이어를 쌓았으며 레이어마다 활성 함수로 LeakyReLU를 넣어줬다. 입력값으로 이미지 크기인 28x28개의 변수를 받은 뒤 레이어의 크기가 28x28에서 1024로, 512로, 256으로 점차 줄어들다. 마지막에는 확률값을 나타내는 숫자 하나를 출력한다.

레이어마다 들어간 드롭아웃(Dropout)은 학습 시에 무작위로 절반의 뉴런을 사용하지 않도록 한다. 이를 통해 모델이 과적합(Overfitting, 오버피팅)되는 것을 방지할 수 있고, 또한 구분자가 생성자보다 지나치게 빨리 학습되는 것도 막을 수 있다. 구분자의 마지막 레이어에서는 출력값을 0과 1 사이로 만들기 위해 활성 함수로 Sigmoid를 넣었다.

## <코드3> GAN의 구분자(Discriminator)


In [26]:
# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력한다.
class Discriminator(nn.Module):

    # 네트워크 구조
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=1024),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout(inplace=False),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
    # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아 이미지가 진짜일 확률을 0 ~ 1 사이로 출력한다.
    def forward(self, inputs):
        inputs = inputs.view(-1, 28 * 28)
        return self.main(inputs)

## <코드4> 생성자와 구분자 객체 만들기

In [27]:
G = Generator()
D = Discriminator()

이제부터는 이렇게 만들어진 네트워크 구조를 학습하는 방법에 대해 알아보자. 학습하기 위해서는 모델을 평가할 수 있어야 한다. 모델의 평가 지표가 좋아지는 방향으로 매개 변수를 업데이트할 것이기 때문이다. 구분자의 출력값은 이미지가 진짜일 확률이고, 이 확률이 얼마나 정답과 가까운지를 측정하기 위해 바이너리 크로스 엔트로피(Binary cross entropy) 손실 함수(loss function)를 사용한다. 이 함수는 구분자가 출력한 확률값이 정답에 가까우면 낮아지고 정답에서 멀면 높아진다. 이 손실 함수의 값을 낮추는 것이 모델 학습의 목표가 된다.

이제 생성자와 구분자의 매개 변수를 업데이트하는 최적화 함수가 각각 하나씩 필요하다. 최적화 기법에는 여러 종류가 있지만 여기서는 가장 널리 쓰이는 기법인 아담(Adam)을 사용했다. 아담은 매개 변수마다 업데이트 속도를 최적으로 조절하는 효율적인 최적화 기법이다.

## <코드5> 손실 함수와 최적화 기법 지정하기

In [28]:
# Binary Cross Entropy loss
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimier = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

모델 학습을 위해서 전체 데이터셋을 여러 번 돌며 매개 변수를 조금씩 업데이트한다. 데이터셋을 한 번 도는 것을 1 에폭(Epoch)이라고 부르는데, 여기서는 100 에폭 동안 학습할 것이다. 각 에폭마다 배치 사이즈(Batch Size)인 60개만큼 데이터를 가져와서 모델을 학습시킨다. MNIST 학습 데이터의 개수가 6만개이니 1에폭마다 1000번씩 학습이 이루어지는 셈이다.

## <코드6> 모델 학습을 위한 반복문

In [ ]:
# 데아터셋을 100번 돌며 학습한다.

for epoch in range(100):
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
    
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)
        # ...

먼저 구분자를 학습시켜보자. 구분자는 진짜 이미지를 입력하면 1에 가까운 확률값을 출력하고, 가짜 데이터를 입력하면 0에 가까운 확률값을 출력해야 한다. 따라서 구분자의 손실 함수는 두 가지의 합으로 이루어진다. 진짜 이미지를 입력했을 때의 출력값과 1과의 차이, 그리고 가짜 이미지를 입력했을 때의 출력값과 0과의 차이, 두 경우의 합이 구분자의 손실 함수다. 이 손실 함수의 값을 최소화하는 방향으로 구분자의 매개 변수가 업데이트된다.

파이토치에서는 간단한 방법으로 역전파를 통해 계산된 각 변수의 미분 값을 구할 수 있다. 그 상태에서 최적화 함수를 실행시키면 매개 변수가 한번 업데이트된다.

## <코드7> 구분자 학습시키기


In [ ]:

# 데아터셋을 100번 돌며 학습한다.

for epoch in range(100):
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
    
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)

        # ### ### ### 구분자 학습시키기 ### ### ### #
        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.

        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))

        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)

        # 구분자의 출력 값이 정답지인 1에서 멀수록 Loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))

        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)

        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_loss_fake, target_fake)

        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake

        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()

        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()

        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimier.step()

다음으로 생성자를 학습할 차례다. 생성자의 목적은 구분자를 속이는 것이다. 다시 말해 생성자가 만들어낸 가짜 이미지를 구분자에 넣었을 때 출력값이 1에 가깝게 나오도록 해야 한다. 이 값이 1에서 떨어진 정도가 생성자의 손실 함수가 되고, 이를 최소화 시키도록 생성자를 학습시키게 된다.

## <코드8> 생성자 학습시키기

In [34]:
# 데아터셋을 100번 돌며 학습한다.

# 시각화를 위해 사용할 고정된 노이즈 벡터
fixed_z = Variable(torch.randn((5 * 5, 100)))  # We will create 25 images


for epoch in range(100):
    # 한 번에 batch_size만큼 데이터를 가져온다.
    for i, (real_data, _) in enumerate(dataloader):
        batch_size = real_data.size(0)
    
    # 데이터를 파이토치의 변수로 변환한다.
        real_data = Variable(real_data)

        # ### ### ### 구분자 학습시키기 ### ### ### #
        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.

        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))

        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)

        # 구분자의 출력 값이 정답지인 1에서 멀수록 Loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))

        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)

        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)

        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake

        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()

        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()

        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimier.step()
        
        # ### ### ### 생성자 학습시키기 ### ### ### 성
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        if torch.cuda.is_available():
            z = z.cuda()

        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)

        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)

        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 Loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)

        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()

        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()

        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()


        if i % 500 == 0:  # Adjust the interval to your preference
            G.eval()  # Set the generator to evaluation mode
            save_image(G(fixed_z).view(-1, 1, 28, 28),
                       f'generated_samples/epoch_{epoch}_batch_{i}.png',
                       nrow=5, normalize=True)  # Save the generated images
            G.train()  # Set the generator back to training mode

        if i % 100 == 0:  # Adjust the interval to your preference
            print(f"Epoch: {epoch}, Batch: {i}, D Loss: {D_loss.item()}, G Loss: {G_loss.item()}")



Epoch: 0, Batch: 0, D Loss: 0.3382036089897156, G Loss: 2.695267677307129
Epoch: 0, Batch: 100, D Loss: 0.5041618347167969, G Loss: 3.301342725753784
Epoch: 0, Batch: 200, D Loss: 0.6896781921386719, G Loss: 2.6689257621765137
Epoch: 0, Batch: 300, D Loss: 0.4603595435619354, G Loss: 1.5370315313339233
Epoch: 0, Batch: 400, D Loss: 0.67864590883255, G Loss: 2.502753734588623
Epoch: 0, Batch: 500, D Loss: 0.5591709613800049, G Loss: 1.5592728853225708
Epoch: 0, Batch: 600, D Loss: 0.3569903075695038, G Loss: 2.7076470851898193
Epoch: 0, Batch: 700, D Loss: 0.4840846657752991, G Loss: 2.5027434825897217
Epoch: 0, Batch: 800, D Loss: 0.5758324861526489, G Loss: 1.991543173789978
Epoch: 0, Batch: 900, D Loss: 0.6301521062850952, G Loss: 3.2881109714508057
Epoch: 1, Batch: 0, D Loss: 0.5475175380706787, G Loss: 2.8206799030303955
Epoch: 1, Batch: 100, D Loss: 0.8374646306037903, G Loss: 2.2299935817718506
Epoch: 1, Batch: 200, D Loss: 0.512529194355011, G Loss: 1.6790850162506104
Epoch: 1, 